# Import Libraries

In [381]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import scipy.stats as stats
import datetime

# Step 1: Import Superfund Sites Datasets and Match Superfund Sites to the Residing Cities

In [382]:
superfund = pd.ExcelFile('superfund.xlsx')
print(superfund.sheet_names)
df=pd.read_excel('superfund.xlsx')
for items in superfund.sheet_names[1:]:
    dfnew=pd.read_excel("superfund.xlsx",sheet_name=items)
    df=pd.concat([df,dfnew])
superfund_df = df
county_list = pd.read_excel("..\Public Data\VARIABLE for coder X.xlsx")


['Region 01', 'Region 02', 'Region 03', 'Region 04', 'Region 05', 'Region 06', 'Region 07', 'Region 08', 'Region 09', 'Region 10']


In [383]:
city_to_county = pd.read_csv(".\city_to_county.csv",encoding='latin-1')
fips_data = pd.read_csv("..\Public Data\state_and_county_fips_master.csv")

In [384]:
superfund_df.head(4)

,site_id,site_epa_id,site_name,rstate_code,rfed_fclty_code,rpm,osc,site_ifms_ssid_code,rnpl_status_desc,fr_npl_update_nmbr,...,ou_id,rat_act_code,rat_name,ralt_desc,raa_desc,act_actl_strt_date,act_actl_cmpltn_date,curr_amt,prior_amt,ou_name
0,100255,CTD980732333,BARKHAMSTED-NEW HARTFORD LANDFILL,CT,No,"Silva, Almerinda","TAGLIAFERRO, DEAN",01B8,Currently on the Final NPL,10.0,...,NaN,DS001,DISCOVERY,EPA Fund-Financed,NaN,NaN,1/1/1981 00:00:00,0,0,SITEWIDE
1,100255,CTD980732333,BARKHAMSTED-NEW HARTFORD LANDFILL,CT,No,"Silva, Almerinda","TAGLIAFERRO, DEAN",01B8,Currently on the Final NPL,10.0,...,NaN,RS001,REMOVAL ASSESSMENT,EPA Fund-Financed,NaN,4/13/1989 00:00:00,4/29/1989 00:00:00,0,0,SITEWIDE
2,100255,CTD980732333,BARKHAMSTED-NEW HARTFORD LANDFILL,CT,No,"Silva, Almerinda","TAGLIAFERRO, DEAN",01B8,Currently on the Final NPL,10.0,...,NaN,RS002,REMOVAL ASSESSMENT,EPA Fund-Financed,NaN,6/14/1990 00:00:00,6/14/1990 00:00:00,0,0,SITEWIDE
3,100255,CTD980732333,BARKHAMSTED-NEW HARTFORD LANDFILL,CT,No,"Silva, Almerinda","TAGLIAFERRO, DEAN",01B8,Currently on the Final NPL,10.0,...,NaN,RS003,REMOVAL ASSESSMENT,EPA Fund-Financed,NaN,8/14/1991 00:00:00,8/14/1991 00:00:00,0,0,SITEWIDE


In [385]:
superfund_df["act_actl_strt_date"] = pd.to_datetime(superfund_df["act_actl_strt_date"])
superfund_df["act_actl_cmpltn_date"] = pd.to_datetime(superfund_df["act_actl_cmpltn_date"])
superfund_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75974 entries, 0 to 4226
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   site_id               75974 non-null  int64         
 1   site_epa_id           75974 non-null  object        
 2   site_name             75974 non-null  object        
 3   rstate_code           75974 non-null  object        
 4   rfed_fclty_code       75974 non-null  object        
 5   rpm                   71387 non-null  object        
 6   osc                   27379 non-null  object        
 7   site_ifms_ssid_code   75809 non-null  object        
 8   rnpl_status_desc      75974 non-null  object        
 9   fr_npl_update_nmbr    56264 non-null  float64       
 10  rsitinc_desc          62341 non-null  object        
 11  rsite_section_name    75974 non-null  object        
 12  site_city_name        75974 non-null  object        
 13  ou_sort_id       

In [386]:
processed_superfund_df = superfund_df.groupby(["site_epa_id","rstate_code","site_city_name"]).agg({"act_actl_cmpltn_date":"min",
                                                                  "act_actl_strt_date":"min"}).reset_index(drop=False)

In [387]:
processed_superfund_df.sort_values(by=["act_actl_cmpltn_date","act_actl_strt_date"])

,site_epa_id,rstate_code,site_city_name,act_actl_cmpltn_date,act_actl_strt_date
917,NJD002385664,NJ,VINELAND,1966-12-01,1983-08-01
163,CAT080012826,CA,MIRA LOMA,1969-03-01,1972-01-01
1093,NYD010959757,NY,COLD SPRINGS,1971-09-01,1982-05-01
944,NJD070281175,NJ,PLEASANTVILLE,1972-01-01,1976-06-01
1469,TND980729172,TN,CHATTANOOGA,1973-01-01,1987-03-28
...,...,...,...,...,...
1678,WAN001002907,WA,BREMERTON,2010-08-30,2010-08-30
236,FLD000623041,FL,JACKSONVILLE,2010-12-01,2010-08-11
1405,PRN000206452,PR,COROZAL,2010-12-06,2012-03-20
1170,NYN000206456,NY,LOCKPORT,2011-03-10,2011-10-17


In [388]:
processed_superfund_df["city_state"]=processed_superfund_df.apply(lambda x: x["site_city_name"]+", "+x["rstate_code"]+", USA",axis=1)

In [389]:
processed_superfund_df

,site_epa_id,rstate_code,site_city_name,act_actl_cmpltn_date,act_actl_strt_date,city_state
0,AK0001897602,AK,THORNE BAY,1997-03-25,1998-11-09,"THORNE BAY, AK, USA"
1,AK1570028646,AK,FAIRBANKS,1981-06-01,1988-10-07,"FAIRBANKS, AK, USA"
2,AK4170024323,AK,ADAK,1981-06-01,1992-01-06,"ADAK, AK, USA"
3,AK6210022426,AK,FORT WAINWRIGHT,1987-08-20,1988-10-14,"FORT WAINWRIGHT, AK, USA"
4,AK6214522157,AK,ANCHORAGE,1991-03-12,1992-06-30,"ANCHORAGE, AK, USA"
...,...,...,...,...,...,...
1734,WVD988798401,WV,VIENNA,1986-01-13,1992-09-15,"VIENNA, WV, USA"
1735,WVSFN0305428,WV,RAVENSWOOD,1998-11-20,1999-01-20,"RAVENSWOOD, WV, USA"
1736,WY5571924179,WY,CHEYENNE,1987-02-10,1989-07-14,"CHEYENNE, WY, USA"
1737,WYD061112470,WY,LARAMIE,1979-10-01,1983-07-01,"LARAMIE, WY, USA"


### The Following Dataset Includes the County Information. The County Information is Retrieved Using Google Geocoding API. More Detailed Process is in EPLRA2_City_to_County.ipynb.

In [390]:
city_to_county

,Unnamed: 0,city_state,county_name,county,lat,lon
0,0.0,"BARKHAMSTED, CT, USA","<[OK] Google - Geocode [Barkhamsted, CT, USA]>",Litchfield County,41.929263,-72.913990
1,1.0,"BEACON FALLS, CT, USA","<[OK] Google - Geocode [Beacon Falls, CT, USA]>",New Haven County,41.442645,-73.061761
2,2.0,"EAST WINDSOR, CT, USA","<[OK] Google - Geocode [East Windsor, CT, USA]>",Hartford County,41.916136,-72.557880
3,3.0,"CHESHIRE, CT, USA","<[OK] Google - Geocode [Cheshire, CT, USA]>",New Haven County,41.508367,-72.910620
4,4.0,"DURHAM, CT, USA","<[OK] Google - Geocode [Durham, CT 06422, USA]>",Middlesex County,41.481782,-72.681258
...,...,...,...,...,...,...
1367,1366.0,"MAPLE VALLEY, WA, USA","<[OK] Google - Geocode [Maple Valley, WA 98038...",King County,47.390340,-122.045359
1368,1367.0,"LOOMIS, WA, USA","<[OK] Google - Geocode [Loomis, WA, USA]>",Okanogan County,48.820975,-119.633400
1369,1368.0,"BRUSH PRAIRIE, WA, USA","<[OK] Google - Geocode [Brush Prairie, WA, USA]>",Clark County,45.732894,-122.546486
1370,1369.0,"MARYSVILLE, WA, USA","<[OK] Google - Geocode [Marysville, WA, USA]>",Snohomish County,48.051764,-122.177082


### Merge County Data to Superfund Sites Data

In [391]:
combined_superfund_df = processed_superfund_df.merge(city_to_county,on=["city_state"],how="left")
combined_superfund_df

,site_epa_id,rstate_code,site_city_name,act_actl_cmpltn_date,act_actl_strt_date,city_state,Unnamed: 0,county_name,county,lat,lon
0,AK0001897602,AK,THORNE BAY,1997-03-25,1998-11-09,"THORNE BAY, AK, USA",1311.0,"<[OK] Google - Geocode [Thorne Bay, AK, USA]>",Prince of Wales-Hyder Census Area,55.686767,-132.523204
1,AK1570028646,AK,FAIRBANKS,1981-06-01,1988-10-07,"FAIRBANKS, AK, USA",1308.0,"<[OK] Google - Geocode [Fairbanks, AK, USA]>",Fairbanks North Star Borough,64.840051,-147.719976
2,AK4170024323,AK,ADAK,1981-06-01,1992-01-06,"ADAK, AK, USA",1307.0,"<[OK] Google - Geocode [Adak, AK, USA]>",Aleutians West Census Area,51.866798,-176.638535
3,AK6210022426,AK,FORT WAINWRIGHT,1987-08-20,1988-10-14,"FORT WAINWRIGHT, AK, USA",1310.0,"<[OK] Google - Geocode [Fort Wainwright, Fairb...",Fairbanks North Star Borough,64.827787,-147.642915
4,AK6214522157,AK,ANCHORAGE,1991-03-12,1992-06-30,"ANCHORAGE, AK, USA",1309.0,"<[OK] Google - Geocode [Anchorage, AK, USA]>",Anchorage Municipality,61.217576,-149.899678
...,...,...,...,...,...,...,...,...,...,...,...
1735,WVD988798401,WV,VIENNA,1986-01-13,1992-09-15,"VIENNA, WV, USA",529.0,"<[OK] Google - Geocode [Vienna, WV, USA]>",Wood County,39.327019,-81.548458
1736,WVSFN0305428,WV,RAVENSWOOD,1998-11-20,1999-01-20,"RAVENSWOOD, WV, USA",528.0,"<[OK] Google - Geocode [Ravenswood, WV 26164, ...",Jackson County,38.948137,-81.760965
1737,WY5571924179,WY,CHEYENNE,1987-02-10,1989-07-14,"CHEYENNE, WY, USA",1204.0,"<[OK] Google - Geocode [Cheyenne, WY, USA]>",Laramie County,41.139981,-104.820246
1738,WYD061112470,WY,LARAMIE,1979-10-01,1983-07-01,"LARAMIE, WY, USA",1203.0,"<[OK] Google - Geocode [Laramie, WY, USA]>",Albany County,41.311367,-105.591101


### Drop the Ones that are not Properly Matched

In [392]:
combined_superfund_df.iloc[list(np.where(combined_superfund_df["county"].isna()))[0],:]

,site_epa_id,rstate_code,site_city_name,act_actl_cmpltn_date,act_actl_strt_date,city_state,Unnamed: 0,county_name,county,lat,lon
41,ASD980637656,AS,PAGO PAGO,1981-07-01,1981-08-01,"PAGO PAGO, AS, USA",1206.0,<[OK] Google - Geocode #2 results>,NaN,37.090240,-95.712891
308,FMD980637987,FM,PALAU,1981-10-01,1981-08-01,"PALAU, FM, USA",1298.0,<[OK] Google - Geocode #2 results>,NaN,37.090240,-95.712891
331,GU6571999519,GU,YIGO,1985-11-01,1992-04-01,"YIGO, GU, USA",1299.0,"<[OK] Google - Geocode [Yigo, Guam]>",NaN,13.563963,144.906134
780,MPD980798318,MP,GARAPAN,1981-03-01,1981-08-01,"GARAPAN, MP, USA",1305.0,<[OK] Google - Geocode #2 results>,NaN,37.090240,-95.712891
834,NCD980602163,NC,210 MILES OF ROADS,1979-08-01,1982-05-25,"210 MILES OF ROADS, NC, USA",666.0,"<[OK] Google - Geocode [NC-210, North Carolina...",NaN,34.937115,-78.611174
1140,NYD980763841,NY,HUDSON RIVER,1983-07-01,1983-08-01,"HUDSON RIVER, NY, USA",263.0,"<[OK] Google - Geocode [New York, NY, USA]>",NaN,40.712775,-74.005973


In [393]:
combined_superfund_df = combined_superfund_df.drop(np.where(combined_superfund_df["county"].isna())[0])

In [394]:
combined_superfund_df

,site_epa_id,rstate_code,site_city_name,act_actl_cmpltn_date,act_actl_strt_date,city_state,Unnamed: 0,county_name,county,lat,lon
0,AK0001897602,AK,THORNE BAY,1997-03-25,1998-11-09,"THORNE BAY, AK, USA",1311.0,"<[OK] Google - Geocode [Thorne Bay, AK, USA]>",Prince of Wales-Hyder Census Area,55.686767,-132.523204
1,AK1570028646,AK,FAIRBANKS,1981-06-01,1988-10-07,"FAIRBANKS, AK, USA",1308.0,"<[OK] Google - Geocode [Fairbanks, AK, USA]>",Fairbanks North Star Borough,64.840051,-147.719976
2,AK4170024323,AK,ADAK,1981-06-01,1992-01-06,"ADAK, AK, USA",1307.0,"<[OK] Google - Geocode [Adak, AK, USA]>",Aleutians West Census Area,51.866798,-176.638535
3,AK6210022426,AK,FORT WAINWRIGHT,1987-08-20,1988-10-14,"FORT WAINWRIGHT, AK, USA",1310.0,"<[OK] Google - Geocode [Fort Wainwright, Fairb...",Fairbanks North Star Borough,64.827787,-147.642915
4,AK6214522157,AK,ANCHORAGE,1991-03-12,1992-06-30,"ANCHORAGE, AK, USA",1309.0,"<[OK] Google - Geocode [Anchorage, AK, USA]>",Anchorage Municipality,61.217576,-149.899678
...,...,...,...,...,...,...,...,...,...,...,...
1735,WVD988798401,WV,VIENNA,1986-01-13,1992-09-15,"VIENNA, WV, USA",529.0,"<[OK] Google - Geocode [Vienna, WV, USA]>",Wood County,39.327019,-81.548458
1736,WVSFN0305428,WV,RAVENSWOOD,1998-11-20,1999-01-20,"RAVENSWOOD, WV, USA",528.0,"<[OK] Google - Geocode [Ravenswood, WV 26164, ...",Jackson County,38.948137,-81.760965
1737,WY5571924179,WY,CHEYENNE,1987-02-10,1989-07-14,"CHEYENNE, WY, USA",1204.0,"<[OK] Google - Geocode [Cheyenne, WY, USA]>",Laramie County,41.139981,-104.820246
1738,WYD061112470,WY,LARAMIE,1979-10-01,1983-07-01,"LARAMIE, WY, USA",1203.0,"<[OK] Google - Geocode [Laramie, WY, USA]>",Albany County,41.311367,-105.591101


### Convert State and County Names to Join the Superfund df to County_List df

In [395]:
import us

In [396]:
def state_abbr_to_name (x):
    ## This function converts state abbrievations to the proper name. Eg. OK->Oklahoma.
    try:
        r = us.states.lookup(x)
    except:
        print(x)
    if r:
        try:
            return r.name
        except:
            print(x)
    else:
        if x=="DC":
            return "District Of Columbia"
        else:
            print(x)

def county_name_converter(x):
    x=x.split(" ")
    x=[i for i in x if i!="County"]
    return " ".join(x)    

In [397]:
combined_superfund_df["rstate_code"]=combined_superfund_df["rstate_code"].apply(lambda x: state_abbr_to_name(x))
combined_superfund_df=combined_superfund_df.rename(columns={"rstate_code":"STATE",
                                      "county":"ALLCOUNTIES"})

In [398]:
county_list

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME
0,Alabama,G0100010,41.0,1,10,Autauga,NaN
1,Alabama,G0100030,41.0,1,30,Baldwin,NaN
2,Alabama,G0100050,41.0,1,50,Barbour,NaN
3,Alabama,G0100070,41.0,1,70,Bibb,NaN
4,Alabama,G0100090,41.0,1,90,Blount,NaN
...,...,...,...,...,...,...,...
3253,Wyoming,G5500650,68.0,56,390,Teton,NaN
3254,Wyoming,G5500670,68.0,56,410,Uinta,NaN
3255,Wyoming,G5500690,68.0,56,430,Washakie,NaN
3256,Wyoming,G5500710,68.0,56,450,Weston,NaN


In [399]:
combined_superfund_df

,site_epa_id,STATE,site_city_name,act_actl_cmpltn_date,act_actl_strt_date,city_state,Unnamed: 0,county_name,ALLCOUNTIES,lat,lon
0,AK0001897602,Alaska,THORNE BAY,1997-03-25,1998-11-09,"THORNE BAY, AK, USA",1311.0,"<[OK] Google - Geocode [Thorne Bay, AK, USA]>",Prince of Wales-Hyder Census Area,55.686767,-132.523204
1,AK1570028646,Alaska,FAIRBANKS,1981-06-01,1988-10-07,"FAIRBANKS, AK, USA",1308.0,"<[OK] Google - Geocode [Fairbanks, AK, USA]>",Fairbanks North Star Borough,64.840051,-147.719976
2,AK4170024323,Alaska,ADAK,1981-06-01,1992-01-06,"ADAK, AK, USA",1307.0,"<[OK] Google - Geocode [Adak, AK, USA]>",Aleutians West Census Area,51.866798,-176.638535
3,AK6210022426,Alaska,FORT WAINWRIGHT,1987-08-20,1988-10-14,"FORT WAINWRIGHT, AK, USA",1310.0,"<[OK] Google - Geocode [Fort Wainwright, Fairb...",Fairbanks North Star Borough,64.827787,-147.642915
4,AK6214522157,Alaska,ANCHORAGE,1991-03-12,1992-06-30,"ANCHORAGE, AK, USA",1309.0,"<[OK] Google - Geocode [Anchorage, AK, USA]>",Anchorage Municipality,61.217576,-149.899678
...,...,...,...,...,...,...,...,...,...,...,...
1735,WVD988798401,West Virginia,VIENNA,1986-01-13,1992-09-15,"VIENNA, WV, USA",529.0,"<[OK] Google - Geocode [Vienna, WV, USA]>",Wood County,39.327019,-81.548458
1736,WVSFN0305428,West Virginia,RAVENSWOOD,1998-11-20,1999-01-20,"RAVENSWOOD, WV, USA",528.0,"<[OK] Google - Geocode [Ravenswood, WV 26164, ...",Jackson County,38.948137,-81.760965
1737,WY5571924179,Wyoming,CHEYENNE,1987-02-10,1989-07-14,"CHEYENNE, WY, USA",1204.0,"<[OK] Google - Geocode [Cheyenne, WY, USA]>",Laramie County,41.139981,-104.820246
1738,WYD061112470,Wyoming,LARAMIE,1979-10-01,1983-07-01,"LARAMIE, WY, USA",1203.0,"<[OK] Google - Geocode [Laramie, WY, USA]>",Albany County,41.311367,-105.591101


In [400]:
def county_name_transformer(x:str,dropped_words):
    # This function converts county name to a "cleaner" form by 1)converting all chars to lower 2)remove the words given by dropped_words 3)remove some special symbols (".","'"). 
    # Eg. if dropped_words = ["county"], DuPage -> dupage, DuPage County -> dupage.  
    x=x.strip()
    x=x.split(" ")
    x=[str.lower(i) for i in x if str.lower(i) not in dropped_words]
    x=[str.replace(i,".","") for i in x]
    x=[str.replace(i,"'","") for i in x]
    x="".join(x)
    return x

In [401]:
county_list["county_transformed"] = county_list["ALLCOUNTIES"].apply(lambda x: county_name_transformer(x,["county","census" ,"area" ,"borough" ,"parish","municipality"]))
county_list.head(3)

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,county_transformed
0,Alabama,G0100010,41.0,1,10,Autauga,NaN,autauga
1,Alabama,G0100030,41.0,1,30,Baldwin,NaN,baldwin
2,Alabama,G0100050,41.0,1,50,Barbour,NaN,barbour


In [402]:
combined_superfund_df["county_transformed"]=combined_superfund_df["ALLCOUNTIES"].apply(lambda x: county_name_transformer(x,["county","census" ,"area" ,"borough" ,"parish","municipality"]))

In [403]:
combined_superfund_df.head(3)

,site_epa_id,STATE,site_city_name,act_actl_cmpltn_date,act_actl_strt_date,city_state,Unnamed: 0,county_name,ALLCOUNTIES,lat,lon,county_transformed
0,AK0001897602,Alaska,THORNE BAY,1997-03-25,1998-11-09,"THORNE BAY, AK, USA",1311.0,"<[OK] Google - Geocode [Thorne Bay, AK, USA]>",Prince of Wales-Hyder Census Area,55.686767,-132.523204,princeofwales-hyder
1,AK1570028646,Alaska,FAIRBANKS,1981-06-01,1988-10-07,"FAIRBANKS, AK, USA",1308.0,"<[OK] Google - Geocode [Fairbanks, AK, USA]>",Fairbanks North Star Borough,64.840051,-147.719976,fairbanksnorthstar
2,AK4170024323,Alaska,ADAK,1981-06-01,1992-01-06,"ADAK, AK, USA",1307.0,"<[OK] Google - Geocode [Adak, AK, USA]>",Aleutians West Census Area,51.866798,-176.638535,aleutianswest


In [404]:
combined_superfund_df = combined_superfund_df.merge(county_list,on=["county_transformed","STATE"],how="left")
combined_superfund_df[combined_superfund_df["ALLCOUNTIES_y"].isna()]

,site_epa_id,STATE,site_city_name,act_actl_cmpltn_date,act_actl_strt_date,city_state,Unnamed: 0,county_name,ALLCOUNTIES_x,lat,lon,county_transformed,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES_y,VARIABLE NAME
329,GUD980637649,Guam,AGANA,1979-11-01,1981-08-01,"AGANA, GU, USA",1300.0,"<[OK] Google - Geocode [HagÃ¥tÃ±a, Guam]>",Honolulu County,13.476928,144.754285,honolulu,NaN,NaN,NaN,NaN,NaN,NaN
1593,VID980651095,Virgin Islands,CHRISTIANSTED,1989-01-31,1986-05-30,"CHRISTIANSTED, VI, USA",344.0,"<[OK] Google - Geocode [Christiansted, St Croi...",Christiansted,17.746640,-64.703198,christiansted,NaN,NaN,NaN,NaN,NaN,NaN
1594,VID982272569,Virgin Islands,TUTU,1987-08-01,1987-01-15,"TUTU, VI, USA",345.0,"<[OK] Google - Geocode [Tutu, Anna's Retreat, ...",Tutu,18.338843,-64.886531,tutu,NaN,NaN,NaN,NaN,NaN,NaN


In [405]:
combined_superfund_df = combined_superfund_df.dropna(subset=["ALLCOUNTIES_y"])

In [406]:
combined_superfund_df = combined_superfund_df.loc[:,["site_epa_id","act_actl_cmpltn_date","act_actl_strt_date","STATEFIPS","COUNTYFIPS"]]
combined_superfund_df

,site_epa_id,act_actl_cmpltn_date,act_actl_strt_date,STATEFIPS,COUNTYFIPS
0,AK0001897602,1997-03-25,1998-11-09,2.0,198.0
1,AK1570028646,1981-06-01,1988-10-07,2.0,90.0
2,AK4170024323,1981-06-01,1992-01-06,2.0,16.0
3,AK6210022426,1987-08-20,1988-10-14,2.0,90.0
4,AK6214522157,1991-03-12,1992-06-30,2.0,20.0
...,...,...,...,...,...
1729,WVD988798401,1986-01-13,1992-09-15,54.0,1070.0
1730,WVSFN0305428,1998-11-20,1999-01-20,54.0,350.0
1731,WY5571924179,1987-02-10,1989-07-14,56.0,210.0
1732,WYD061112470,1979-10-01,1983-07-01,56.0,10.0


## Find the Earlest Date When Sites Are Active (Active Means There is a Record in the NPL Dataset)

In [407]:
combined_superfund_df["Earliest_active_date"]= combined_superfund_df.apply( lambda x: x["act_actl_cmpltn_date"] if x["act_actl_cmpltn_date"]<x["act_actl_strt_date"] else x["act_actl_strt_date"], axis=1) 

In [408]:
combined_superfund_df

,site_epa_id,act_actl_cmpltn_date,act_actl_strt_date,STATEFIPS,COUNTYFIPS,Earliest_active_date
0,AK0001897602,1997-03-25,1998-11-09,2.0,198.0,1997-03-25
1,AK1570028646,1981-06-01,1988-10-07,2.0,90.0,1981-06-01
2,AK4170024323,1981-06-01,1992-01-06,2.0,16.0,1981-06-01
3,AK6210022426,1987-08-20,1988-10-14,2.0,90.0,1987-08-20
4,AK6214522157,1991-03-12,1992-06-30,2.0,20.0,1991-03-12
...,...,...,...,...,...,...
1729,WVD988798401,1986-01-13,1992-09-15,54.0,1070.0,1986-01-13
1730,WVSFN0305428,1998-11-20,1999-01-20,54.0,350.0,1998-11-20
1731,WY5571924179,1987-02-10,1989-07-14,56.0,210.0,1987-02-10
1732,WYD061112470,1979-10-01,1983-07-01,56.0,10.0,1979-10-01


In [409]:
combined_superfund_df=combined_superfund_df.groupby(by=["STATEFIPS","COUNTYFIPS"]).agg({"Earliest_active_date":"min"}).reset_index(drop=False)
combined_superfund_df

,STATEFIPS,COUNTYFIPS,Earliest_active_date
0,1.0,30.0,1982-08-01
1,1.0,130.0,1980-12-01
2,1.0,150.0,1982-11-01
3,1.0,670.0,1995-09-09
4,1.0,730.0,1984-01-06
...,...,...,...
765,72.0,137.0,1984-09-15
766,72.0,141.0,1981-07-01
767,72.0,143.0,1982-07-01
768,72.0,145.0,1988-09-19


In [410]:
county_list=county_list.merge(combined_superfund_df,on=["STATEFIPS","COUNTYFIPS"],how="left")
county_list

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,county_transformed,Earliest_active_date
0,Alabama,G0100010,41.0,1,10,Autauga,NaN,autauga,NaT
1,Alabama,G0100030,41.0,1,30,Baldwin,NaN,baldwin,1982-08-01
2,Alabama,G0100050,41.0,1,50,Barbour,NaN,barbour,NaT
3,Alabama,G0100070,41.0,1,70,Bibb,NaN,bibb,NaT
4,Alabama,G0100090,41.0,1,90,Blount,NaN,blount,NaT
...,...,...,...,...,...,...,...,...,...
3253,Wyoming,G5500650,68.0,56,390,Teton,NaN,teton,NaT
3254,Wyoming,G5500670,68.0,56,410,Uinta,NaN,uinta,NaT
3255,Wyoming,G5500690,68.0,56,430,Washakie,NaN,washakie,NaT
3256,Wyoming,G5500710,68.0,56,450,Weston,NaN,weston,NaT


In [411]:
county_list.shape[0]-np.sum(county_list.isna())

STATE                   3258
GISJOIN                 3258
STATEICP                3166
STATEFIPS               3258
COUNTYFIPS              3258
ALLCOUNTIES             3258
VARIABLE NAME              0
county_transformed      3258
Earliest_active_date     770
dtype: int64

In [412]:
county_list[county_list["Earliest_active_date"].apply(lambda x: x.year<=1990)].count()

STATE                   693
GISJOIN                 693
STATEICP                680
STATEFIPS               693
COUNTYFIPS              693
ALLCOUNTIES             693
VARIABLE NAME             0
county_transformed      693
Earliest_active_date    693
dtype: int64

## Part 2: Working with IPUMS 1990 Census Data, Find Counties with More Black Residents

### First, Import IPUMS 1990 5% Census Data, and See Which PUMA Regions has Greater Black Pop, and We'll Later Mark all the Counties within These PUMA Regions with Greater Black Pop.

In [413]:
from ipumspy import readers, ddi

ddi_codebook = readers.read_ipums_ddi(r"usa_00005.xml")
ipums_df = readers.read_microdata(ddi_codebook, r"usa_00005.dat")

c:\Users\longy\AppData\Local\Programs\Python\Python39\lib\site-packages\ipumspy\readers.py:49: CitationWarning: Use of data from IPUMS is subject to conditions including that users should cite the data appropriately.
See the `ipums_conditions` attribute of this codebook for terms of use.
See the `ipums_citation` attribute of this codebook for the appropriate citation.
  warnings.warn(


In [414]:
ipums_df

,STATEFIP,COUNTYFIP,PUMA,RACE,RACED
0,2,20,101,1,100
1,2,20,101,1,100
2,2,20,101,1,100
3,2,20,101,1,100
4,2,20,101,7,700
...,...,...,...,...,...
12501041,56,0,400,1,100
12501042,56,0,400,1,100
12501043,56,0,400,1,100
12501044,56,0,400,1,100


In [415]:
ipums_df["Black"]=ipums_df["RACED"]==200 ##Code 200 means the sample is black

In [416]:
ipums_df

,STATEFIP,COUNTYFIP,PUMA,RACE,RACED,Black
0,2,20,101,1,100,False
1,2,20,101,1,100,False
2,2,20,101,1,100,False
3,2,20,101,1,100,False
4,2,20,101,7,700,False
...,...,...,...,...,...,...
12501041,56,0,400,1,100,False
12501042,56,0,400,1,100,False
12501043,56,0,400,1,100,False
12501044,56,0,400,1,100,False


In [417]:
region_sum = ipums_df.groupby(["STATEFIP","PUMA"]).agg({"Black":"sum","RACED":"count"}).reset_index(drop=False)
region_sum.rename(columns={"RACED":"Pop"})

,STATEFIP,PUMA,Black,Pop
0,1,100,817,6167
1,1,200,382,6940
2,1,300,2309,8301
3,1,400,1552,5570
4,1,500,2868,4815
...,...,...,...,...
1721,55,2400,18,6844
1722,56,100,79,5475
1723,56,200,17,5247
1724,56,300,27,5988


### Convert PUMA to County 

In [418]:
puma = pd.read_excel("1990_PUMAs_5pct (1).xlsx")

In [419]:
puma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35475 entries, 0 to 35474
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   State             35475 non-null  int64  
 1   PUMA              35475 non-null  int64  
 2   Type of area      35475 non-null  object 
 3   County name       35475 non-null  object 
 4   Area population   35475 non-null  int64  
 5   County FIPS code  35475 non-null  int64  
 6   MCD               32004 non-null  float64
 7   Place             27124 non-null  float64
 8   MSA/PMSA          35475 non-null  int64  
 9   PUMA population   35475 non-null  int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 2.7+ MB


In [420]:
puma = puma[["State","PUMA","County name","County FIPS code"]]

In [421]:
puma

,State,PUMA,County name,County FIPS code
0,1,100,Colbert County,33
1,1,100,Lauderdale County,77
2,1,200,Franklin County,59
3,1,200,Walker County,127
4,1,200,Winston County,133
...,...,...,...,...
35470,56,400,Johnson County,19
35471,56,400,Park County,29
35472,56,400,Sheridan County,33
35473,56,400,Washakie County,43


In [422]:
puma.loc[puma["County FIPS code"].astype(str).apply(lambda x: x.endswith("0")),"State"].value_counts()

24    203
29    114
2      71
51     34
32      1
Name: State, dtype: int64

In [423]:
len(county_list.loc[county_list["COUNTYFIPS"].astype(str).apply(lambda x: x.endswith("0")),"STATEFIPS"].value_counts())

50

In [424]:
puma["converted_countyname"]=puma["County name"].apply(lambda x: county_name_transformer(x,["county","census" ,"area" ,"borough" ,"parish","municipality","(part)"]))

In [425]:
puma.loc[puma["County FIPS code"].astype(int)%10 ==0,"State"].value_counts()

24    203
29    114
2      71
51     34
32      1
Name: State, dtype: int64

In [426]:
def fips_converter(state, county):
    ## This is a function that converts the county FIPS code in county_list to the normal FIPS code as in puma dataset.
    irregular = [2]
    regualr = [i for i in range(57) if i not in irregular]
    if state in irregular:
        return county
    else:
        if int(county)%10 == 0:
            return str(int(county/10))
        else:
            return county

In [427]:
puma["County FIPS code"] = puma["County FIPS code"].astype(str)

In [428]:
county_list["converted_fips"]=county_list.apply(lambda x: fips_converter(x["STATEFIPS"], x["COUNTYFIPS"]),axis=1)

In [429]:
county_list.head(3)

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,county_transformed,Earliest_active_date,converted_fips
0,Alabama,G0100010,41.0,1,10,Autauga,NaN,autauga,NaT,1
1,Alabama,G0100030,41.0,1,30,Baldwin,NaN,baldwin,1982-08-01,3
2,Alabama,G0100050,41.0,1,50,Barbour,NaN,barbour,NaT,5


In [430]:
temp = pd.merge(left=county_list, right=puma,
               left_on=["STATEFIPS","converted_fips"],right_on=["State","County FIPS code"],
               how="left")


In [431]:
np.sum(temp.isna())

STATE                       0
GISJOIN                     0
STATEICP                   92
STATEFIPS                   0
COUNTYFIPS                  0
ALLCOUNTIES                 0
VARIABLE NAME           35316
county_transformed          0
Earliest_active_date     7337
converted_fips              0
State                     149
PUMA                      149
County name               149
County FIPS code          149
converted_countyname      149
dtype: int64

In [432]:
temp.loc[(temp["PUMA"].isna()) & (~temp["Earliest_active_date"].isna()),:]

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,county_transformed,Earliest_active_date,converted_fips,State,PUMA,County name,County FIPS code,converted_countyname
385,Alaska,G0100290,81.0,2,16,Aleutians West,NaN,aleutianswest,1981-06-01,16,NaN,NaN,NaN,NaN,NaN
386,Alaska,G0100310,81.0,2,20,Anchorage,NaN,anchorage,1981-03-01,20,NaN,NaN,NaN,NaN,NaN
396,Alaska,G0100510,81.0,2,90,Fairbanks North Star,NaN,fairbanksnorthstar,1980-06-01,90,NaN,NaN,NaN,NaN,NaN
415,Alaska,G0100890,81.0,2,198,Prince of Wales-Hyder Census Area,NaN,princeofwales-hyder,1997-03-25,198,NaN,NaN,NaN,NaN,NaN
7781,Florida,G0801190,NaN,12,86,Miami-Dade County,NaN,miami-dade,1978-12-01,86,NaN,NaN,NaN,NaN,NaN
8912,Hawaii,G1302070,82.0,15,35,Honolulu,NaN,honolulu,1980-04-01,35,NaN,NaN,NaN,NaN,NaN
29305,Puerto Rico,G5500850,NaN,72,13,Arecibo Municipio,NaN,arecibomunicipio,1989-04-21,13,NaN,NaN,NaN,NaN,NaN
29307,Puerto Rico,G5500890,NaN,72,17,Barceloneta Municipio,NaN,barcelonetamunicipio,1981-05-01,17,NaN,NaN,NaN,NaN,NaN
29310,Puerto Rico,G5500950,NaN,72,23,Cabo Rojo Municipio,NaN,caborojomunicipio,2007-03-15,23,NaN,NaN,NaN,NaN,NaN
29311,Puerto Rico,G5500970,NaN,72,25,Caguas Municipio,NaN,caguasmunicipio,2010-01-13,25,NaN,NaN,NaN,NaN,NaN


### There are Counties that Cannot be Matched. The Reason Might be that Some Counties Changed From 1990s. We'll Correct Them Manually.

In [454]:
ddi_codebook = readers.read_ipums_ddi(r"usa_pr.xml")
pr_df = readers.read_microdata(ddi_codebook, r"usa_pr.dat")

c:\Users\longy\AppData\Local\Programs\Python\Python39\lib\site-packages\ipumspy\readers.py:49: CitationWarning: Use of data from IPUMS is subject to conditions including that users should cite the data appropriately.
See the `ipums_conditions` attribute of this codebook for terms of use.
See the `ipums_citation` attribute of this codebook for the appropriate citation.
  warnings.warn(


In [456]:
pr_df[(pr_df["BPL"]=="600") | (pr_df["MBPL"]=="600") | (pr_df["FBPL"]=="600")]

,BPL,BPLD,MBPL,MBPLD,FBPL,FBPLD


In [458]:
np.sum(pr_df[pr_df=="600"].isna())

BPL      177655
BPLD     177655
MBPL     177655
MBPLD    177655
FBPL     177655
FBPLD    177655
dtype: int64

In [433]:
mannual_corrections = pd.read_csv("Missing_puma_fix.csv")
mannual_corrections=mannual_corrections.drop(columns=["Unnamed: 0"])

In [434]:
mannual_corrections

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,county_transformed,Earliest_active_date,c_cfips_x,converted_fips,State,PUMA,County name,County FIPS code,converted_countyname,c_cfips_y
0,Alaska,G0100290,81.0,2,16,Aleutians West,NaN,aleutianswest,1981/6/1,16,16.0,NaN,300,NaN,NaN,NaN,NaN
1,Alaska,G0100310,81.0,2,20,Anchorage,NaN,anchorage,1981/3/1,2,20.0,NaN,101,NaN,NaN,NaN,NaN
2,Alaska,G0100510,81.0,2,90,Fairbanks North Star,NaN,fairbanksnorthstar,1980/6/1,9,90.0,NaN,200,NaN,NaN,NaN,NaN
3,Alaska,G0100890,81.0,2,198,Prince of Wales-Hyder Census Area,NaN,princeofwales-hyder,1997/3/25,198,198.0,NaN,300,NaN,NaN,NaN,NaN
4,Florida,G0801190,NaN,12,86,Miami-Dade County,NaN,miami-dade,1978/12/1,86,NaN,NaN,3601,NaN,NaN,NaN,NaN
5,Florida,G0801190,NaN,12,86,Miami-Dade County,NaN,miami-dade,1978/12/1,86,NaN,NaN,3602,NaN,NaN,NaN,NaN
6,Hawaii,G1302070,82.0,15,35,Honolulu,NaN,honolulu,1980/4/1,35,NaN,NaN,301,NaN,NaN,NaN,NaN
7,Hawaii,G1302070,82.0,15,35,Honolulu,NaN,honolulu,1980/4/1,35,NaN,NaN,302,NaN,NaN,NaN,NaN
8,Hawaii,G1302070,82.0,15,35,Honolulu,NaN,honolulu,1980/4/1,35,NaN,NaN,303,NaN,NaN,NaN,NaN
9,Hawaii,G1302070,82.0,15,35,Honolulu,NaN,honolulu,1980/4/1,35,NaN,NaN,304,NaN,NaN,NaN,NaN


In [435]:
temp=temp.drop(np.where([temp["County FIPS code"].isna() & ~temp["Earliest_active_date"].isna() & (temp["STATE"]!="Puerto Rico")])[1])
temp=pd.concat([temp,mannual_corrections])

In [436]:
temp

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,county_transformed,Earliest_active_date,converted_fips,State,PUMA,County name,County FIPS code,converted_countyname,c_cfips_x,c_cfips_y
0,Alabama,G0100010,41.0,1,10,Autauga,NaN,autauga,NaT,1,1.0,1500.0,Autauga County,1,autauga,NaN,NaN
1,Alabama,G0100030,41.0,1,30,Baldwin,NaN,baldwin,1982-08-01 00:00:00,3,1.0,800.0,Baldwin County,3,baldwin,NaN,NaN
2,Alabama,G0100050,41.0,1,50,Barbour,NaN,barbour,NaT,5,1.0,1100.0,Barbour County,5,barbour,NaN,NaN
3,Alabama,G0100070,41.0,1,70,Bibb,NaN,bibb,NaT,7,1.0,300.0,Bibb County,7,bibb,NaN,NaN
4,Alabama,G0100090,41.0,1,90,Blount,NaN,blount,NaT,9,1.0,2100.0,Blount County,9,blount,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,Hawaii,G1302070,82.0,15,35,Honolulu,NaN,honolulu,1980/4/1,NaN,NaN,303.0,NaN,NaN,NaN,35.0,NaN
9,Hawaii,G1302070,82.0,15,35,Honolulu,NaN,honolulu,1980/4/1,NaN,NaN,304.0,NaN,NaN,NaN,35.0,NaN
10,Hawaii,G1302070,82.0,15,35,Honolulu,NaN,honolulu,1980/4/1,NaN,NaN,305.0,NaN,NaN,NaN,35.0,NaN
11,Hawaii,G1302070,82.0,15,35,Honolulu,NaN,honolulu,1980/4/1,NaN,NaN,306.0,NaN,NaN,NaN,35.0,NaN


In [437]:
temp = temp[["STATEFIPS","COUNTYFIPS","ALLCOUNTIES","Earliest_active_date","PUMA"]]
temp["PUMA"]=temp["PUMA"].astype("Int64")
np.sum(temp.isna())

C:\Users\longy\AppData\Local\Temp/ipykernel_5604/2232239049.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["PUMA"]=temp["PUMA"].astype("Int64")


STATEFIPS                  0
COUNTYFIPS                 0
ALLCOUNTIES                0
Earliest_active_date    7337
PUMA                     143
dtype: int64

In [438]:
temp

,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,Earliest_active_date,PUMA
0,1,10,Autauga,NaT,1500
1,1,30,Baldwin,1982-08-01 00:00:00,800
2,1,50,Barbour,NaT,1100
3,1,70,Bibb,NaT,300
4,1,90,Blount,NaT,2100
...,...,...,...,...,...
8,15,35,Honolulu,1980/4/1,303
9,15,35,Honolulu,1980/4/1,304
10,15,35,Honolulu,1980/4/1,305
11,15,35,Honolulu,1980/4/1,306


In [439]:
region_sum

,STATEFIP,PUMA,Black,RACED
0,1,100,817,6167
1,1,200,382,6940
2,1,300,2309,8301
3,1,400,1552,5570
4,1,500,2868,4815
...,...,...,...,...
1721,55,2400,18,6844
1722,56,100,79,5475
1723,56,200,17,5247
1724,56,300,27,5988


### After Matching Counties with Corresponding PUMA Regions, Assign PUMA Demographic Data to Counties.

In [440]:
combined_census_df= temp.merge(region_sum,left_on=["STATEFIPS","PUMA"],right_on=["STATEFIP","PUMA"],how="left")

In [441]:
np.sum(combined_census_df.isna())

STATEFIPS                  0
COUNTYFIPS                 0
ALLCOUNTIES                0
Earliest_active_date    7337
PUMA                     143
STATEFIP                 143
Black                    143
RACED                    143
dtype: int64

In [442]:
combined_census_df["black_ratio"]=combined_census_df["Black"]/combined_census_df["RACED"]

## Step 3: Create Label: 1=Counties with Greater or Equal Proportion of Black Residents (According to PUMA Estimation) Than National Average in 1990 that Has at Least 1 Superfund Site  

### First, We Estimate National Black Proportion using the 5% Census Data.

In [443]:
national_black_avg=np.sum(ipums_df["Black"])/ipums_df.shape[0]

In [444]:
combined_census_df["minority_county"]=(combined_census_df["black_ratio"]>=national_black_avg)
combined_census_df["minority_county"]=combined_census_df["minority_county"].fillna(False)
combined_census_df["minority_county"]=combined_census_df["minority_county"].astype("int64")

combined_census_df["superfund_sites_active"]= combined_census_df["Earliest_active_date"]

In [445]:
combined_census_df

,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,Earliest_active_date,PUMA,STATEFIP,Black,RACED,black_ratio,minority_county,superfund_sites_active
0,1,10,Autauga,NaT,1500,1,1046,5856.0,0.17862,1,NaT
1,1,30,Baldwin,1982-08-01 00:00:00,800,1,965,6480.0,0.14892,1,1982-08-01 00:00:00
2,1,50,Barbour,NaT,1100,1,2772,10381.0,0.267026,1,NaT
3,1,70,Bibb,NaT,300,1,2309,8301.0,0.278159,1,NaT
4,1,90,Blount,NaT,2100,1,94,6362.0,0.014775,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...
35318,15,35,Honolulu,1980/4/1,303,15,28,5386.0,0.005199,0,1980/4/1
35319,15,35,Honolulu,1980/4/1,304,15,57,6015.0,0.009476,0,1980/4/1
35320,15,35,Honolulu,1980/4/1,305,15,236,7186.0,0.032842,0,1980/4/1
35321,15,35,Honolulu,1980/4/1,306,15,350,5450.0,0.06422,0,1980/4/1


In [446]:
combined_census_df=combined_census_df[["STATEFIPS","COUNTYFIPS","ALLCOUNTIES","Earliest_active_date","PUMA","minority_county"]]

In [447]:
labeled=combined_census_df.groupby(["STATEFIPS","COUNTYFIPS"]).agg({"Earliest_active_date":"min",
                                                            "minority_county":"max"}).reset_index(drop=False)

C:\Users\longy\AppData\Local\Temp/ipykernel_5604/3504594551.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  labeled=combined_census_df.groupby(["STATEFIPS","COUNTYFIPS"]).agg({"Earliest_active_date":"min",


In [448]:
labeled

,STATEFIPS,COUNTYFIPS,Earliest_active_date,minority_county
0,1,10,NaT,1
1,1,30,1982-08-01 00:00:00,1
2,1,50,NaT,1
3,1,70,NaT,1
4,1,90,NaT,0
...,...,...,...,...
3252,72,145,1988-09-19 00:00:00,0
3253,72,147,2002-05-01 00:00:00,0
3254,72,149,NaT,0
3255,72,151,NaT,0


In [449]:
np.sum(labeled["minority_county"])

819

In [450]:
labeled["label"]=(labeled["Earliest_active_date"].apply(lambda x: x.year<=1990)) & (labeled["minority_county"]==1)
labeled["label"]=labeled["label"].astype(int)
labeled.head(3)

,STATEFIPS,COUNTYFIPS,Earliest_active_date,minority_county,label
0,1,10,NaT,1,0
1,1,30,1982-08-01 00:00:00,1,1
2,1,50,NaT,1,0


In [451]:
labeled=labeled[["STATEFIPS","COUNTYFIPS","label"]]
county_list=county_list.merge(labeled,on=["COUNTYFIPS","STATEFIPS"],how="left")


In [452]:
county_list=county_list.drop(columns=["county_transformed","Earliest_active_date","converted_fips"])
county_list

,STATE,GISJOIN,STATEICP,STATEFIPS,COUNTYFIPS,ALLCOUNTIES,VARIABLE NAME,label
0,Alabama,G0100010,41.0,1,10,Autauga,NaN,0
1,Alabama,G0100030,41.0,1,30,Baldwin,NaN,1
2,Alabama,G0100050,41.0,1,50,Barbour,NaN,0
3,Alabama,G0100070,41.0,1,70,Bibb,NaN,0
4,Alabama,G0100090,41.0,1,90,Blount,NaN,0
...,...,...,...,...,...,...,...,...
3253,Wyoming,G5500650,68.0,56,390,Teton,NaN,0
3254,Wyoming,G5500670,68.0,56,410,Uinta,NaN,0
3255,Wyoming,G5500690,68.0,56,430,Washakie,NaN,0
3256,Wyoming,G5500710,68.0,56,450,Weston,NaN,0
